In [1]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from numpy import argmax
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from tqdm import tqdm
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import GridSearchCV
import collections
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import ShuffleSplit
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.inspection import permutation_importance
import pandas as pd

In [2]:
import numpy as np

In [3]:
human_lombardo = pd.read_csv("../data/human_smiles_r_v3.csv")
human_lombardo = human_lombardo[human_lombardo["smiles_r"]!="Cannot_do"].reset_index(drop=True)

In [4]:
human_lombardo=  human_lombardo[["smiles_r","human_VDss_L_kg","human_CL_mL_min_kg", "human_fup", "human_mrt", "human_thalf"]]

for endpoint in ["human_VDss_L_kg","human_CL_mL_min_kg", "human_mrt", "human_thalf" ]:
    print("Transformed endpoint to log base 10")
    human_lombardo[endpoint] = np.log10(human_lombardo[endpoint])
human_lombardo

Transformed endpoint to log base 10
Transformed endpoint to log base 10
Transformed endpoint to log base 10
Transformed endpoint to log base 10


,smiles_r,human_VDss_L_kg,human_CL_mL_min_kg,human_fup,human_mrt,human_thalf
0,C#CCC(Cc1cnc2nc(N)nc(N)c2n1)c1ccc(C(=O)NC(CCC(...,0.006466,0.638489,0.33,0.568202,1.176091
1,C#CCCCC(=O)c1cc(C(C)(C)C)c([O-])c(C(C)(C)C)c1,1.079181,0.949390,NaN,1.348305,1.705008
2,C#CCN(Cc1cc2c(=O)nc(C)[n-]c2cc1C)c1ccc(C(=O)NC...,-0.275724,-0.537602,0.01,1.482588,1.894870
3,C#CC[NH+](C)C(C)Cc1ccccc1,0.278754,1.301030,0.13,0.204120,0.113943
4,C#C[C@@]1(O)[C@@H](CO)O[C@H](n2ccc(N)nc2=O)[C@...,0.041393,0.204120,NaN,1.068186,1.037426
...,...,...,...,...,...,...
1278,c1cc2c(c(N3CC[NH2+]CC3)c1)OCCO2,0.556303,0.903090,NaN,1.021189,0.913814
1279,c1ccc(C2([NH+]3CCCCC3)CCCCC2)cc1,0.838849,0.707570,0.35,1.361728,1.204120
1280,c1ccc(C2(c3ccccc3)CC2C2=[NH+]CCN2)cc1,0.612784,0.934498,0.50,0.892095,0.863323
1281,c1ccc(CN(CC2=[NH+]CCN2)c2ccccc2)cc1,0.591065,1.293804,NaN,0.537819,0.359835


In [5]:
human_lombardo = human_lombardo.groupby('smiles_r').median().reset_index()
human_lombardo

,smiles_r,human_VDss_L_kg,human_CL_mL_min_kg,human_fup,human_mrt,human_thalf
0,C#CCC(Cc1cnc2nc(N)nc(N)c2n1)c1ccc(C(=O)NC(CCC(...,0.006466,0.638489,0.33,0.568202,1.176091
1,C#CCCCC(=O)c1cc(C(C)(C)C)c([O-])c(C(C)(C)C)c1,1.079181,0.949390,NaN,1.348305,1.705008
2,C#CCN(Cc1cc2c(=O)nc(C)[n-]c2cc1C)c1ccc(C(=O)NC...,-0.275724,-0.537602,0.01,1.482588,1.894870
3,C#CC[NH+](C)C(C)Cc1ccccc1,0.278754,1.301030,0.13,0.204120,0.113943
4,C#C[C@@]1(O)[C@@H](CO)O[C@H](n2ccc(N)nc2=O)[C@...,0.041393,0.204120,NaN,1.068186,1.037426
...,...,...,...,...,...,...
1278,c1cc2c(c(N3CC[NH2+]CC3)c1)OCCO2,0.556303,0.903090,NaN,1.021189,0.913814
1279,c1ccc(C2([NH+]3CCCCC3)CCCCC2)cc1,0.838849,0.707570,0.35,1.361728,1.204120
1280,c1ccc(C2(c3ccccc3)CC2C2=[NH+]CCN2)cc1,0.612784,0.934498,0.50,0.892095,0.863323
1281,c1ccc(CN(CC2=[NH+]CCN2)c2ccccc2)cc1,0.591065,1.293804,NaN,0.537819,0.359835


In [6]:
human_lombardo.to_csv("Train_data_log_transformed.csv", index=False)

In [7]:
human_smiles_r = human_lombardo[["smiles_r"]]
human_smiles_r

,smiles_r
0,C#CCC(Cc1cnc2nc(N)nc(N)c2n1)c1ccc(C(=O)NC(CCC(...
1,C#CCCCC(=O)c1cc(C(C)(C)C)c([O-])c(C(C)(C)C)c1
2,C#CCN(Cc1cc2c(=O)nc(C)[n-]c2cc1C)c1ccc(C(=O)NC...
3,C#CC[NH+](C)C(C)Cc1ccccc1
4,C#C[C@@]1(O)[C@@H](CO)O[C@H](n2ccc(N)nc2=O)[C@...
...,...
1278,c1cc2c(c(N3CC[NH2+]CC3)c1)OCCO2
1279,c1ccc(C2([NH+]3CCCCC3)CCCCC2)cc1
1280,c1ccc(C2(c3ccccc3)CC2C2=[NH+]CCN2)cc1
1281,c1ccc(CN(CC2=[NH+]CCN2)c2ccccc2)cc1


In [8]:
from sklearn.feature_selection import VarianceThreshold
from itertools import compress

In [9]:
from rdkit import Chem
from mordred import Calculator, descriptors

# create descriptor calculator with all descriptors
calc = Calculator(descriptors, ignore_3D=True)

print(len(calc.descriptors))

Ser_Mol = human_lombardo['smiles_r'].apply(Chem.MolFromSmiles)

# as pandas
Mordred_table=  calc.pandas(Ser_Mol)

1613


  1%|▌                                        | 18/1283 [00:01<02:04, 10.14it/s]

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 20%|███████▋                               | 252/1283 [00:02<00:05, 194.56it/s]

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, ax

...

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 89%|█████████████████████████████████▋    | 1136/1283 [00:07<00:00, 275.63it/s]

/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████████████████████████████████| 1283/1283 [00:07<00:00, 163.91it/s]


/home/ss2686/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [10]:
Mordred_table = Mordred_table.astype('float')
Mordred_table['smiles_r'] = human_lombardo['smiles_r']
Mordred_table

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,smiles_r
0,26.726479,20.199330,2.0,0.0,43.712035,2.420516,4.841033,43.712035,1.248915,4.459804,...,71.132315,475.161514,8.485027,4344.0,54.0,176.0,202.0,13.083333,7.805556,C#CCC(Cc1cnc2nc(N)nc(N)c2n1)c1ccc(C(=O)NC(CCC(...
1,16.484101,15.092007,1.0,0.0,24.810570,2.476291,4.952581,24.810570,1.127753,3.981521,...,56.111877,299.201654,6.106156,1036.0,34.0,110.0,125.0,11.180556,4.750000,C#CCCCC(=O)c1cc(C(C)(C)C)c([O-])c(C(C)(C)C)c1
2,30.181133,23.132813,4.0,0.0,49.053928,2.449016,4.898018,49.053928,1.257793,4.581296,...,89.010877,529.176450,8.675024,5657.0,62.0,202.0,236.0,13.805556,8.611111,C#CCN(Cc1cc2c(=O)nc(C)[n-]c2cc1C)c1ccc(C(=O)NC...
3,10.077834,9.321842,0.0,1.0,17.214150,2.226811,4.453621,17.214150,1.229582,3.514311,...,44.372474,188.143376,5.879480,337.0,16.0,62.0,67.0,5.333333,3.361111,C#CC[NH+](C)C(C)Cc1ccccc1
4,14.384684,13.322169,0.0,0.0,23.623048,2.530742,4.897943,23.623048,1.243318,3.870813,...,67.418049,267.085521,8.346423,654.0,34.0,100.0,122.0,8.229167,4.291667,C#C[C@@]1(O)[C@@H](CO)O[C@H](n2ccc(N)nc2=O)[C@...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278,12.606602,10.129166,0.0,1.0,22.084432,2.412947,4.825894,22.084432,1.380277,3.723828,...,48.647579,221.128454,6.700862,412.0,24.0,84.0,99.0,3.444444,3.583333,c1cc2c(c(N3CC[NH2+]CC3)c1)OCCO2
1279,14.018917,11.675935,0.0,1.0,24.741115,2.478172,4.956344,24.741115,1.374506,3.833767,...,51.418286,244.205976,5.550136,549.0,29.0,94.0,112.0,4.034722,4.083333,c1ccc(C2([NH+]3CCCCC3)CCCCC2)cc1
1280,16.038187,14.420804,0.0,2.0,27.701443,2.658726,5.024986,27.701443,1.385072,3.997563,...,75.593926,263.154275,6.747546,751.0,30.0,112.0,139.0,4.256944,4.402778,c1ccc(C2(c3ccccc3)CC2C2=[NH+]CCN2)cc1
1281,15.515909,12.987974,0.0,2.0,26.938755,2.326623,4.635214,26.938755,1.346938,3.923960,...,66.060685,266.165174,6.654129,829.0,24.0,100.0,113.0,4.444444,4.527778,c1ccc(CN(CC2=[NH+]CCN2)c2ccccc2)cc1


In [11]:
len(Mordred_table.columns[Mordred_table.isna().any()].tolist())

441

In [12]:
Ser_Mol = human_lombardo['smiles_r'].apply(Chem.MolFromSmiles)

import numpy as np
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprintAsBitVect

Morgan_fingerprint= Ser_Mol.apply(GetMorganFingerprintAsBitVect, args=(2, 2048))
Morganfingerprint_array  = np.stack(Morgan_fingerprint)

Morgan_collection  = []
for x in np.arange(Morganfingerprint_array.shape[1]): #np.arange plus rapide que range
    x = "Mfp"+str(x)
    Morgan_collection.append(x)

Morganfingerprint_table  = pd.DataFrame(Morganfingerprint_array , columns=Morgan_collection )
Morganfingerprint_table['smiles_r'] = human_lombardo['smiles_r']
Morganfingerprint_table

,Mfp0,Mfp1,Mfp2,Mfp3,Mfp4,Mfp5,Mfp6,Mfp7,Mfp8,Mfp9,...,Mfp2039,Mfp2040,Mfp2041,Mfp2042,Mfp2043,Mfp2044,Mfp2045,Mfp2046,Mfp2047,smiles_r
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,C#CCC(Cc1cnc2nc(N)nc(N)c2n1)c1ccc(C(=O)NC(CCC(...
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,C#CCCCC(=O)c1cc(C(C)(C)C)c([O-])c(C(C)(C)C)c1
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,C#CCN(Cc1cc2c(=O)nc(C)[n-]c2cc1C)c1ccc(C(=O)NC...
3,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,C#CC[NH+](C)C(C)Cc1ccccc1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,C#C[C@@]1(O)[C@@H](CO)O[C@H](n2ccc(N)nc2=O)[C@...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,c1cc2c(c(N3CC[NH2+]CC3)c1)OCCO2
1279,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,c1ccc(C2([NH+]3CCCCC3)CCCCC2)cc1
1280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,c1ccc(C2(c3ccccc3)CC2C2=[NH+]CCN2)cc1
1281,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,c1ccc(CN(CC2=[NH+]CCN2)c2ccccc2)cc1


In [13]:
human_lombardo_mfp = pd.merge(human_lombardo, Morganfingerprint_table)
human_lombardo_mfp_Mordred = pd.merge(human_lombardo_mfp, Mordred_table)
human_lombardo_mfp_Mordred

#human_lombardo_mfp = pd.merge(human_lombardo, Morganfingerprint_table)
#human_lombardo_mfp_Mordred = pd.merge(human_lombardo_mfp, Mordred_table)
#human_lombardo_mfp_Mordred

,smiles_r,human_VDss_L_kg,human_CL_mL_min_kg,human_fup,human_mrt,human_thalf,Mfp0,Mfp1,Mfp2,Mfp3,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,C#CCC(Cc1cnc2nc(N)nc(N)c2n1)c1ccc(C(=O)NC(CCC(...,0.006466,0.638489,0.33,0.568202,1.176091,0,1,0,0,...,10.345445,71.132315,475.161514,8.485027,4344.0,54.0,176.0,202.0,13.083333,7.805556
1,C#CCCCC(=O)c1cc(C(C)(C)C)c([O-])c(C(C)(C)C)c1,1.079181,0.949390,NaN,1.348305,1.705008,0,0,0,0,...,10.048237,56.111877,299.201654,6.106156,1036.0,34.0,110.0,125.0,11.180556,4.750000
2,C#CCN(Cc1cc2c(=O)nc(C)[n-]c2cc1C)c1ccc(C(=O)NC...,-0.275724,-0.537602,0.01,1.482588,1.894870,0,1,0,0,...,10.533295,89.010877,529.176450,8.675024,5657.0,62.0,202.0,236.0,13.805556,8.611111
3,C#CC[NH+](C)C(C)Cc1ccccc1,0.278754,1.301030,0.13,0.204120,0.113943,0,1,0,0,...,8.987072,44.372474,188.143376,5.879480,337.0,16.0,62.0,67.0,5.333333,3.361111
4,C#C[C@@]1(O)[C@@H](CO)O[C@H](n2ccc(N)nc2=O)[C@...,0.041393,0.204120,NaN,1.068186,1.037426,0,0,0,0,...,10.060961,67.418049,267.085521,8.346423,654.0,34.0,100.0,122.0,8.229167,4.291667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278,c1cc2c(c(N3CC[NH2+]CC3)c1)OCCO2,0.556303,0.903090,NaN,1.021189,0.913814,0,0,0,0,...,9.706073,48.647579,221.128454,6.700862,412.0,24.0,84.0,99.0,3.444444,3.583333
1279,c1ccc(C2([NH+]3CCCCC3)CCCCC2)cc1,0.838849,0.707570,0.35,1.361728,1.204120,0,0,1,0,...,9.934065,51.418286,244.205976,5.550136,549.0,29.0,94.0,112.0,4.034722,4.083333
1280,c1ccc(C2(c3ccccc3)CC2C2=[NH+]CCN2)cc1,0.612784,0.934498,0.50,0.892095,0.863323,0,0,0,0,...,10.245906,75.593926,263.154275,6.747546,751.0,30.0,112.0,139.0,4.256944,4.402778
1281,c1ccc(CN(CC2=[NH+]CCN2)c2ccccc2)cc1,0.591065,1.293804,NaN,0.537819,0.359835,0,0,0,0,...,9.582249,66.060685,266.165174,6.654129,829.0,24.0,100.0,113.0,4.444444,4.527778


In [14]:
from sklearn.impute import SimpleImputer
import pickle
def find_median_tables(data, animal):#log human_VDss_L_kg model
    
    #Read columns needed for rat data
    file = open(f"features_mfp_mordred_columns_{animal}_model.txt", "r")
    file_lines = file.read()
    features = file_lines.split("\n")
    features = features[:-1]

    X = data[features]

    return(X)

In [15]:
X = find_median_tables(human_lombardo_mfp_Mordred,"dog")   
X = pd.DataFrame(X.median()).T
X.to_csv("Median_mordred_values_dog_for_artificial_animal_data_mfp_mrd_model.csv", index=False)

In [16]:
X = find_median_tables(human_lombardo_mfp_Mordred,"monkey")   
X = pd.DataFrame(X.median()).T
X.to_csv("Median_mordred_values_monkey_for_artificial_animal_data_mfp_mrd_model.csv", index=False)

In [17]:
X = find_median_tables(human_lombardo_mfp_Mordred,"rat")   
X = pd.DataFrame(X.median()).T
X.to_csv("Median_mordred_values_rat_for_artificial_animal_data_mfp_mrd_model.csv", index=False)

In [18]:
X[X.isna().any(axis=1)]

,nAcid,nBase,SpDiam_A,VE1_A,VR2_A,VR3_A,nSpiro,nBridgehead,nHetero,nH,...,Mfp1915,Mfp1917,Mfp1921,Mfp1928,Mfp1955,Mfp1970,Mfp1980,Mfp1991,Mfp2031,Mfp2038


In [19]:
human_lombardo_mfp_Mordred

,smiles_r,human_VDss_L_kg,human_CL_mL_min_kg,human_fup,human_mrt,human_thalf,Mfp0,Mfp1,Mfp2,Mfp3,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,C#CCC(Cc1cnc2nc(N)nc(N)c2n1)c1ccc(C(=O)NC(CCC(...,0.006466,0.638489,0.33,0.568202,1.176091,0,1,0,0,...,10.345445,71.132315,475.161514,8.485027,4344.0,54.0,176.0,202.0,13.083333,7.805556
1,C#CCCCC(=O)c1cc(C(C)(C)C)c([O-])c(C(C)(C)C)c1,1.079181,0.949390,NaN,1.348305,1.705008,0,0,0,0,...,10.048237,56.111877,299.201654,6.106156,1036.0,34.0,110.0,125.0,11.180556,4.750000
2,C#CCN(Cc1cc2c(=O)nc(C)[n-]c2cc1C)c1ccc(C(=O)NC...,-0.275724,-0.537602,0.01,1.482588,1.894870,0,1,0,0,...,10.533295,89.010877,529.176450,8.675024,5657.0,62.0,202.0,236.0,13.805556,8.611111
3,C#CC[NH+](C)C(C)Cc1ccccc1,0.278754,1.301030,0.13,0.204120,0.113943,0,1,0,0,...,8.987072,44.372474,188.143376,5.879480,337.0,16.0,62.0,67.0,5.333333,3.361111
4,C#C[C@@]1(O)[C@@H](CO)O[C@H](n2ccc(N)nc2=O)[C@...,0.041393,0.204120,NaN,1.068186,1.037426,0,0,0,0,...,10.060961,67.418049,267.085521,8.346423,654.0,34.0,100.0,122.0,8.229167,4.291667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278,c1cc2c(c(N3CC[NH2+]CC3)c1)OCCO2,0.556303,0.903090,NaN,1.021189,0.913814,0,0,0,0,...,9.706073,48.647579,221.128454,6.700862,412.0,24.0,84.0,99.0,3.444444,3.583333
1279,c1ccc(C2([NH+]3CCCCC3)CCCCC2)cc1,0.838849,0.707570,0.35,1.361728,1.204120,0,0,1,0,...,9.934065,51.418286,244.205976,5.550136,549.0,29.0,94.0,112.0,4.034722,4.083333
1280,c1ccc(C2(c3ccccc3)CC2C2=[NH+]CCN2)cc1,0.612784,0.934498,0.50,0.892095,0.863323,0,0,0,0,...,10.245906,75.593926,263.154275,6.747546,751.0,30.0,112.0,139.0,4.256944,4.402778
1281,c1ccc(CN(CC2=[NH+]CCN2)c2ccccc2)cc1,0.591065,1.293804,NaN,0.537819,0.359835,0,0,0,0,...,9.582249,66.060685,266.165174,6.654129,829.0,24.0,100.0,113.0,4.444444,4.527778


In [20]:
#What are the rows with NaNs in the human dataset
X = find_median_tables(human_lombardo_mfp_Mordred,"dog")   
X[X.isna().any(axis=1)]

,nAcid,nBase,SpDiam_A,VE1_A,VR2_A,VR3_A,nBridgehead,nHetero,nH,nC,...,Mfp1921,Mfp1928,Mfp1963,Mfp1970,Mfp1980,Mfp1991,Mfp2010,Mfp2021,Mfp2031,Mfp2038
998,0.0,1.0,2.828427,1.707107,1.121195,0.009035,0.0,2.0,3.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1118,1.0,0.0,4.000000,2.309401,1.921668,1.934128,0.0,4.0,1.0,2.0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
#What are the columns with NaNs in the human dataset
X[X.isna().any(axis=1)][X.columns[X.isna().any()].tolist()]

,AATS4dv,AATS4d,AATS4s,AATS4m,AATS4v,AATS4are,AATS4i,AATSC4dv,AATSC4s,AATSC4m,AATSC4v,GATS4c,GATS4s,GATS4se,Kier3
998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
